# Calculate blockage corrections with BEET and CFD.ML
CFD.ML may be use to derive a blockage correction to a wakes-only, engineering model.
This notebook performs the calculation, and provides a comparisson to the BEET blockage correction results!

In [1]:
import os
import requests
import json
import time
import random
import asyncio
import pandas as pd
from copy import deepcopy

### Export script inputs from WindFarmer
Export the annual energy production calculation API input settings from within your workbook using an in-app script. Edit the file path and call:

```Toolbox.ExportWindFarmerEnergyJson( @"C:\folder\my_aep_calculation_inputs.json" );```

### User inputs
Edit below!

In [2]:
# Exported AEP input settings from your workbook using an in-app script and calling Toolbox.ExportWindFarmerEnergyJson( input_data_file_path ):
AEP_calculation_input_json_file_path = r'..\..\..\DemoData\TheBowl\TheBowl.json'
# 2 Graph Neural Networks are available in Version 2.3
gnn_type = "offshore" # "onshore" / "offshore"

number_of_direction_steps = 180
blockage_correction_application_method = "OnEnergy" #  "OnEnergy" / "OnWindSpeed" options

To access the API you need a authorization token. 
This should be kept secure - and not added to source control, so I'm getting it from an environment variable. See setup instructions for saving your access key as an environment variable documented [here.](https://myworkspace.dnv.com/download/public/renewables/windfarmer/manuals/latest/WebAPI/Introduction/gettingStarted.html)

In [3]:
api_url = 'https://windfarmer.dnv.com/api/v2/'
auth_token = os.environ['WINDFARMER_ACCESS_KEY']
# The token should be passed as an Authorization header. We also need to set the `Content-Type` to let the API know that we're sending JSON data.
headers = {
    'Authorization': f'Bearer {auth_token}',
    'Content-Type': 'application/json'
}

### Call `Status` to check your connection
If this fails to return status 200 then check your access key is saved in the environment variable above

In [4]:
response = requests.get(api_url + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

Response from Status: 200
{"message":"Connection to DNV WindFarmer Services API was successful.","windFarmerServicesAPIVersion":"2.3.0","calculationLibraryVersion":"2.2.31.0"}


### Blockage calculations - call `AnnualEnergyProduction` endpoint

In [5]:
# load the inputs
with open(AEP_calculation_input_json_file_path) as f:
    json_string = f.read()
    json_input = json.loads(json_string)

turbines = []
for farm in json_input["windFarms"]:
    for turbine in farm["turbines"]:
        turbines.append(turbine)
number_of_turbines = len(turbines)
print(f'wind farms in project contain {number_of_turbines} turbines')

wind farms in project contain 110 turbines


Define settings required to make a fast evaluation of CFD.ML and BEET blockage correction efficiencies

In [6]:
json_input= json.loads(json_string)
# general settings for all models
json_input['energyEfficienciesSettings']['numberOfDirectionSectorsForWakeCalculation'] = number_of_direction_steps
# turn off wake models etc. so only considering blockage for speed of computation
json_input['energyEfficienciesSettings']['wakeModel']['wakeModelType'] = 'NoWakeModel'
json_input["energyEfficienciesSettings"]["wakeModel"]["noWakeModel"]["useLargeWindFarmModel"] = False
json_input["energyEfficienciesSettings"]["calculateEfficiencies"] = False
json_input["energyEfficienciesSettings"]["includeHysteresisEffect"] = False
json_input["energyEfficienciesSettings"]["includeTurbineManagement"] = False
json_input["energyEfficienciesSettings"]["calculateIdealYield"] = False
json_input["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"] = {}

beet_stable_inputs = deepcopy(json_input)
beet_stable_inputs['energyEfficienciesSettings']['blockageModel']['beet']['blockageCorrectionApplicationMethod'] = blockage_correction_application_method
beet_stable_inputs['energyEfficienciesSettings']['blockageModel']['blockageModelType'] = "BEET" 
beet_stable_inputs["energyEfficienciesSettings"]["blockageModel"]["beet"]["significantAtmosphericStability"] = True

beet_neutral_unstable_inputs = deepcopy(beet_stable_inputs)
beet_neutral_unstable_inputs["energyEfficienciesSettings"]["blockageModel"]["beet"]["significantAtmosphericStability"] = False

cfdml_inputs = deepcopy(json_input)
cfdml_inputs['energyEfficienciesSettings']['blockageModel']['blockageModelType'] = "CFDML" 
cfdml_inputs["energyEfficienciesSettings"]["blockageModel"]["cfdml"]["cfdmlBlockageWindSpeedDependency"] = "FromBlockageExtrapolationCurve" 
cfdml_inputs['energyEfficienciesSettings']['blockageModel']['cfdml']['blockageCorrectionApplicationMethod'] = blockage_correction_application_method
cfdml_inputs["energyEfficienciesSettings"]["blockageModel"]["cfdml"]["cfdmlSettings"]["gnnType"] = gnn_type
cfdml_inputs["energyEfficienciesSettings"]["blockageModel"]["cfdml"]["cfdmlSettings"]["gnnVersion"] = "1.1" # This calculator is only for version 1

Some methods to make a calls to the WFer Services API and compute the blockage correction regardless of approach.
We call the asynchronous AEP method for farms with > 150 turbines

In [7]:
# Polling for status for AnnualEnergyProductionAsync calculations
async def get_jobstatus( job_id: str) -> (str, str):
    params = {}
    params["jobId"] = job_id
    result =  requests.get(api_url + 'AnnualEnergyProductionAsync', headers=headers, params= params)
    result_json = json.loads(result.content)
    return (result_json['status'], result_json['results'] if 'results' in result_json else None)


# Aynchronous calculations are slower, but reliable for long running calculations as we implement a job queue.
async def call_aep_api_async(api_url, headers, input_data, min_polling_interval_seconds):
    start = time.time()
    job_id_response = requests.post(
        api_url + 'AnnualEnergyProductionAsync', 
        headers=headers,
        json = input_data)    
    print(f'Response {job_id_response.status_code} - {job_id_response.reason} in {time.time() - start:.2f}s')
    # Print the error detail if we haven't receieved a 200 OK response
    if job_id_response.status_code != 202:
        print(json.loads(job_id_response.content)['detail'])
    else:
        job_ID = json.loads (job_id_response.text)["jobId"]
        print ('...Job sumbimitted with ID: ')
        print (job_ID)
        # Poll for status
        status = "PENDING"
        await ( asyncio.sleep(random.random() + 5))
        while(status == 'PENDING' or status == 'RUNNING'):
            (status, results ) = await get_jobstatus( job_ID)
            print(f'...Calculation status @ {time.time() - start:.2f}s: {status}')
            if status == 'FAILED':
                raise Exception("Calculation failed")
            await ( asyncio.sleep(random.random() + min_polling_interval_seconds))
        print(f'{status} in {time.time() - start:.2f}s')
        return results


# Synchronous calculations are faster, but unreliable for large wind farms:
def call_aep_api_sync(api_url, headers, input_data):
    start = time.time()
    response = requests.post(
        api_url + 'AnnualEnergyProduction', 
        headers=headers,
        json = input_data)
    print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')

    if response.status_code == 200:
        results = json.loads(response.content)
        return results
    else:
        # Print the error detail if we haven't receieved a 200 OK response 
        print(json.loads(response.content)['detail'])
        return None
    

# Decide whether to call the synchronous or asynchronous end point based on number of turbines
async def call_aep_api(api_url, headers, input_data):
    turbines = []
    for farm in input_data["windFarms"]:
        for turbine in farm["turbines"]:
            turbines.append(turbine)
    number_of_turbines = len(turbines)
    blockage_model_type = input_data['energyEfficienciesSettings']['blockageModel']['blockageModelType'] 
    if number_of_turbines < 150 or blockage_model_type == "BEET":
        print(f'Calling synchronous AEP API for {blockage_model_type} blockage calculation')
        results = call_aep_api_sync(api_url, headers, input_data)
    else:
        print(f'Calling asynchronous AEP API for {blockage_model_type} blockage calculation')
        time_per_turbine = 600 / 1000
        min_polling_interval_seconds = number_of_turbines * time_per_turbine / 10
        results = await call_aep_api_async(api_url, headers, input_data, min_polling_interval_seconds)
    return results


def get_blockage_efficinecy(results_dict,blockage_correction_application_method):
    blockage_correction_efficiency = -1
    if blockage_correction_application_method == "OnEnergy":
        blockage_correction_efficiency = float(results_dict['weightedBlockageEfficiency'])
    elif blockage_correction_application_method == "OnWindSpeed":
        full_aep_MWh_per_year = sum([float(x['fullAnnualEnergyYield_MWh_per_year']) for x in results_dict['windFarmAepOutputs']])
        gross_aep_MWh_per_year = sum([float(x['grossAnnualEnergyYield_MWh_per_year']) for x in results_dict['windFarmAepOutputs']])
        blockage_correction_efficiency = full_aep_MWh_per_year / gross_aep_MWh_per_year
    else:
        print("blockage_correction_application_method not recognised")
    return blockage_correction_efficiency    

In [8]:
# Make the API calls

beet_stable_results= await call_aep_api( api_url, headers, beet_stable_inputs)
beet_stable_blockage_correction_efficiency = get_blockage_efficinecy(beet_stable_results, blockage_correction_application_method)

beet_neutral_unstable_results= await call_aep_api( api_url, headers, beet_neutral_unstable_inputs)
beet_neutral_unstable_blockage_correction_efficiency = get_blockage_efficinecy(beet_neutral_unstable_results, blockage_correction_application_method)

cfdml_results = await call_aep_api( api_url, headers, cfdml_inputs)
cfdml_blockage_correction_efficiency = get_blockage_efficinecy(cfdml_results, blockage_correction_application_method)


Calling synchronous AEP API for BEET blockage calculation
Response 200 - OK in 2.83s
Calling synchronous AEP API for BEET blockage calculation
Response 200 - OK in 2.57s
Calling synchronous AEP API for CFDML blockage calculation
Response 200 - OK in 29.03s


In [9]:
# Print results:
print(f'Blockage correction application method = {blockage_correction_application_method}')
print(f'Note, the BEET model was derived from RANS CFD simulations approapriate to onshore boundary layers \nCFD.MLv1 has an onshore or offshore option in API v2.3\n')
columns = ['Model', 'Atmospheric stability', 'On/Offshore',  'Blockage correction efficiency [%]']
results_data = [
    ['BEET', 'Stable', 'onshore', beet_stable_blockage_correction_efficiency*100],
    ['BEET', 'Neutral/Unstable', 'onshore', beet_neutral_unstable_blockage_correction_efficiency*100],
    ['CFD.ML', 'Neutral', gnn_type, cfdml_blockage_correction_efficiency*100],
]
results_df = pd.DataFrame(results_data, columns=columns)
results_df

Blockage correction application method = OnEnergy
Note, the BEET model was derived from RANS CFD simulations approapriate to onshore boundary layers 
CFD.ML has an onshore or offshore option in API v2.3



,Model,Atmospheric stability,On/Offshore,Blockage correction efficiency [%]
0,BEET,Stable,onshore,97.118855
1,BEET,Neutral/Unstable,onshore,98.357485
2,CFD.ML,Neutral,offshore,96.705627
